In [1]:
# https://towardsdatascience.com/visualizing-feature-vectors-embeddings-using-pca-and-t-sne-ef157cea3a42
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from matplotlib import cm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import torch.utils.data as td
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision import transforms
from torchvision import models
import os

In [2]:
def count_target_classes(root_dir):
    '''return count of number of images per class'''
    target_classes = {}
    for folder in sorted(os.listdir(root_dir)):
        data_path_for_image_folder = root_dir+ '/'+str(folder) + '/'
        target_classes[str(folder)] = len([image_filename for image_filename in sorted(os.listdir(data_path_for_image_folder))])
    return target_classes

def dataset_loader(path, val_split, test_split, input_size, batch_size, shuffle_test=False):
    
    transform_dict = {'src':  transforms.Compose([transforms.Resize(input_size), 
                                                transforms.RandomHorizontalFlip(),
                                                transforms.RandomVerticalFlip(),
                                                transforms.RandomAdjustSharpness(0.2),
                                                transforms.RandomAutocontrast(),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),])}

    data = datasets.ImageFolder(root=path, transform=transform_dict["src"])

    test_size = int(len(data) * test_split)
    train_size_temp = len(data)- test_size

    train_dataset_temp, test_dataset = td.random_split(data, [train_size_temp, test_size])

    val_size = int(len(train_dataset_temp) * val_split)
    train_size = len(train_dataset_temp) - val_size

    # split validate set from train set: https://stackoverflow.com/questions/13610074/is-there-a-rule-of-thumb-for-how-to-divide-a-dataset-into-training-and-validatio
    train_dataset, val_dataset = td.random_split(train_dataset_temp, [train_size, val_size])
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle_test, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle_test, pin_memory=True)

    return train_loader, val_loader, test_loader

In [5]:
root_ds_path = "C:/Users/Damian/Documents/School/Fall 2022 - Grad School/Comp 6721/Project/Repo"
root_txt_path = "C:/Users/Damian/Documents/School/Fall 2022 - Grad School/Comp 6721/Project/Repo/My Work/Model_Stats"
root_model_path = "C:/Users/Damian/Documents/School/Fall 2022 - Grad School/Comp 6721/Project/Repo/My Work/Saved_Models/"
root_saved_model_path = "C:/Users/Damian/Documents/School/Fall 2022 - Grad School/Comp 6721/Project/Repo/Damians Stuff/Saved_Models/"

ds_dir_dict = {"EuroSat": root_ds_path+"/LandscapeClassification/Datasets/EuroSat/EuroSAT", 
               "Landuse": root_ds_path+"/LandscapeClassification/Datasets/Land-Use Scene Classification/images",
               "SRSI": root_ds_path+"/LandscapeClassification/Datasets/SRSI RSI CB256/data"}

model_dict = {"VGG": [models.vgg16(weights=None), 4096],
              "ResNet": [models.resnet50(weights=None), 2048],
              "EffNet": [models.efficientnet_b0(weights=None), 2048]}

In [8]:
dict_class_count = {}
for ds in ds_dir_dict:
    data_dir = ds_dir_dict[ds]
    dict_class_count[ds] = count_target_classes(data_dir)
    train_loader, val_loader, test_loader = dataset_loader(data_dir, val_split=.2, test_split=0.2, input_size=[64,64], batch_size=32)

In [13]:
model = model_dict["VGG"][0]  # By default, no pre-trained weights are used.
model.fc = nn.Linear(model_dict["VGG"][1], len(dict_class_count["EuroSat"])) # change output parameters to match number of classes
model.load_state_dict(torch.load(root_saved_model_path+"EuroSat_VGG_20.pth"))
named_layers = dict(model.named_modules())
print(named_layers)
# model.eval()
# model = model.cuda()

{'': VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilat